In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import resourcecode
from resourcecode import innosea

%matplotlib widget

# load WEC capture width

In [2]:
INNOSEA_DATA_DIR = Path(innosea.__file__).parent / "Inputs"
capture_width_path = INNOSEA_DATA_DIR / "capture_width.csv"
freq_path = INNOSEA_DATA_DIR / "Frequencies.csv"
pto_data_path = INNOSEA_DATA_DIR / "PTO_values.csv"

In [3]:
capture_width = pd.read_csv(capture_width_path, delimiter=",", header=None)
freq = pd.read_csv(freq_path, delimiter=",", header=None)
pto_values = pd.read_csv(pto_data_path, delimiter=",", header=None)
capture_width.columns = pto_values.values.tolist()[0]
capture_width.index = [val for sublist in freq.values.tolist() for val in sublist]

# load hs/tp data

Loads Hs and Tp time series from cassandra, stores it as a pandas.DataFrame

In [4]:
client = resourcecode.Client()
wave_data = client.get_dataframe_from_criteria(
    """
    {
        "node": 0,
        "start": 961939610,
        "end": 1489903600,
        "parameter": ["hs", "tp"]
    }
    """
)

# create wave spectrum
Creates JONSWAP wave spectrum time series from Hs and Tp time series

In [5]:
freq_vec = capture_width.index
spectrum = innosea.create_wave_spectrum(wave_data, freq_vec)

2054.702760147
358.6240651019998


# set PTO

PTO object, storing capture width, wave spectrum, and computing PTO data such as time series of wave power, absorbed power, mean power, median power, PTO damping


In [ ]:
pto = innosea.PTO(capture_width, spectrum)

# plot time series
Plot PTO results time series in 3 subplots: wave power, absorbed/mean power
    with/without reduction factor, PTO damping with/without reduction factor.
    Power is converted from W to kW, damping from Ns/m to kNs/m.

In [ ]:
# wave power
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 10))
ax1.plot(pto.wave_power.div(1000 * pto.width))
ax1.legend(["Wave power"])
ax1.grid()
ax1.set(xlabel="Time", ylabel="Power (kW/m)")
# absorbed/mean power
all_time_series = [
    pto.power,
    pto.power_no_red,
    pto.mean_power,
    pto.mean_power_no_red,
]
linestyles = ["solid", "dashed", "solid", "dashed"]
for time_series, linestyle in zip(all_time_series, linestyles):
    ax2.plot(time_series.div(1000), linestyle=linestyle)
ax2.legend(
    [
        "Absorbed power (with reduction factor)",
        "Absorbed power (without reduction factor)",
        "Mean power (with reduction factor)",
        "Mean power (without reduction factor)",
    ]
)
ax2.grid()
ax2.set(xlabel="Time", ylabel="Power (kW)")
# PTO damping
all_time_series = [pto.pto_damp, pto.pto_damp_no_red]
for time_series, linestyle in zip(all_time_series, linestyles):
    ax3.plot(time_series.div(1000), linestyle=linestyle)
ax3.legend(
    [
        "PTO damping (with reduction factor)",
        "PTO damping (without reduction factor)",
    ]
)
ax3.grid()
ax3.set(xlabel="Time", ylabel="Damping (kN.s/m)")

# plot_cumulative_power

Plot PTO cumulative power. Power is converted from W to kW

In [ ]:
power_kw = pto.power.div(1000)
# cumulative power
cumulative_power_kw = pto.cumulative_power
power_ordered = pto.power.sort_values(by=0)
index = [x / 1000 for x in power_ordered[0]]
cumulative_power_kw.index = index
# mean power
mean_power_kw = pto.mean_power[0][pto.times[0]] / 1000
# median power
median_power_kw = pto.median_power[0][pto.times[0]] / 1000
# power occurrences, cumulative power, mean and median power
ax = power_kw.plot.hist(
    bins=len(pto.capture_width.columns) * 5,
    legend=False,
    weights=np.ones_like(power_kw[power_kw.columns[0]]) * 100.0 / len(power_kw),
)
ax1 = ax.twinx()
cumulative_power_kw.plot(ax=ax1, legend=False, color="r")
ax.grid()
ax.set_xlabel("WEC Power (kW)")
ax.set_ylabel("Occurrence (%)")
ax1.set_ylabel("Normed Cumulative Production (%)")
line_mean = plt.axvline(x=mean_power_kw, color="y")
line_median = plt.axvline(x=median_power_kw, color="orange")
ax.legend([line_mean, line_median], ["Mean power", "Median power"])

# plot PTO histogram

In [ ]:
pto_damp_kn = pto.pto_damp / 1000
ax = pto_damp_kn.plot.hist(
    bins=len(pto.capture_width.columns),
    legend=False,
    weights=np.ones_like(pto_damp_kn[pto_damp_kn.columns[0]])
    * 100.0
    / len(pto_damp_kn),
)
ax.grid()
ax.set_xlabel("PTO damping (kN.s/m)")
ax.set_ylabel("Occurrence (%)")